In [14]:
import statsmodels.api as sm
import pandas as pd #librería para manejo de datos
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

datos_f1=pd.read_csv('datos/Datos F1_nuevo.csv', sep=';',decimal=',') 
X=datos_f1.values[:,9:15]
X = X.astype('float64')
y=datos_f1.values[:,0]
y = y.astype('float64')

mco1 = sm.OLS(y, sm.add_constant(X)).fit()

mco1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     39.26
Date:                Mon, 22 Nov 2021   Prob (F-statistic):           2.50e-18
Time:                        10:34:00   Log-Likelihood:                -197.25
No. Observations:                  63   AIC:                             408.5
Df Residuals:                      56   BIC:                             423.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.1237     11.157      2.252      0.028       2.773      47.475
x1            11.5783      1.226      9.448      0.000       9.123      14.033
x2             2.2551      0.368      6.121      0.000       1.517       2.993
x3            -1.1888      0.681     -1.745      0.086      -2.554       0.176
x4             0.4657      0.468      0.995      0.324      -0.472       1.404
x5            -0.0982      0.064     -1.526      0.133      -0.227       0.031
x6            -0.1179      0.304     -0.388      0.700      -0.727       0.491
==============================================================================
Omnibus:                       31.939   Durbin-Watson:                   1.452
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               96.371
Skew:                           1.435   Prob(JB):                     1.18e-21
Kurtosis:                       8.337   Cond. No.                         912.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""